In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load Dataset
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = boston.target  # Add target variable

# Discretize AGE variable
def categorize_age(age):
    if age <= 35:
        return '35 and younger'
    elif 35 < age <= 70:
        return 'Between 35 and 70'
    else:
        return '70 and older'

df['AGE_Group'] = df['AGE'].apply(categorize_age)

# --- Visualization ---
plt.figure(figsize=(15,10))

# Boxplot for MEDV
plt.subplot(2,3,1)
sns.boxplot(y=df['MEDV'])
plt.title('Boxplot of Median Home Value')

# Bar plot for Charles River (CHAS)
plt.subplot(2,3,2)
sns.countplot(x=df['CHAS'])
plt.title('Number of Homes near Charles River')

# Boxplot of MEDV vs AGE_Group
plt.subplot(2,3,3)
sns.boxplot(x=df['AGE_Group'], y=df['MEDV'], data=df)
plt.title('Median Home Value by Age Group')

# Scatter Plot of NOX vs INDUS
plt.subplot(2,3,4)
sns.scatterplot(x=df['NOX'], y=df['INDUS'])
plt.title('NOX vs INDUS')

# Histogram for PTRATIO
plt.subplot(2,3,5)
sns.histplot(df['PTRATIO'], bins=15, kde=True)
plt.title('Distribution of Pupil-Teacher Ratio')

plt.tight_layout()
plt.show()

# --- Statistical Tests ---
# 1. T-test: Is there a difference in MEDV for homes near Charles River?
chas_1 = df[df['CHAS'] == 1]['MEDV']
chas_0 = df[df['CHAS'] == 0]['MEDV']
t_stat, p_value = stats.ttest_ind(chas_1, chas_0)
print(f'T-test result: t-stat = {t_stat:.3f}, p-value = {p_value:.3f}')

# 2. ANOVA: Difference in MEDV across AGE groups
anova_result = stats.f_oneway(
    df[df['AGE_Group'] == '35 and younger']['MEDV'],
    df[df['AGE_Group'] == 'Between 35 and 70']['MEDV'],
    df[df['AGE_Group'] == '70 and older']['MEDV']
)
print(f'ANOVA result: F-stat = {anova_result.statistic:.3f}, p-value = {anova_result.pvalue:.3f}')

# 3. Pearson Correlation: NOX vs INDUS
corr, p_value = stats.pearsonr(df['NOX'], df['INDUS'])
print(f'Correlation between NOX and INDUS: r = {corr:.3f}, p-value = {p_value:.3f}')

# 4. Regression: Impact of DIS on MEDV
X = sm.add_constant(df['DIS'])  # Adding constant for intercept
model = sm.OLS(df['MEDV'], X).fit()
print(model.summary())


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: